In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
# run from root
!pwd

from os import listdir
from os.path import isfile, join
# create filenames for pd.read_csv()
files = [f for f in listdir('./csv') if isfile(join('./csv', f))]

# specify year to go back to
# example below is just one year
# we used 3 years which takes time to preprocess below
from_year = 2017
# use to cut down years pulled
files_cut = list(sorted([f for f in files if (int(f[:2]) >= from_year%2000)]))
print(files_cut)

/home/rjh336/nyc17_ds13/student_submissions/projects/01-benson/benson_team_6
['170408.csv', '170415.csv', '170422.csv', '170429.csv', '170506.csv', '170513.csv', '170520.csv', '170527.csv', '170603.csv', '170610.csv', '170617.csv', '170624.csv', '170701.csv']


In [3]:
# Create initial DataFrame
dfs = pd.DataFrame()
for file in files_cut:
    print("Loading {}".format(str("./csv/"+file)))
    df = pd.read_csv(str("./csv/"+file))
    dfs = dfs.append(df)
    
dfs.info()

Loading ./csv/170408.csv
Loading ./csv/170415.csv
Loading ./csv/170422.csv
Loading ./csv/170429.csv
Loading ./csv/170506.csv
Loading ./csv/170513.csv
Loading ./csv/170520.csv
Loading ./csv/170527.csv
Loading ./csv/170603.csv
Loading ./csv/170610.csv
Loading ./csv/170617.csv
Loading ./csv/170624.csv
Loading ./csv/170701.csv
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2565187 entries, 0 to 197420
Data columns (total 12 columns):
Unnamed: 0                                                              int64
C/A                                                                     object
UNIT                                                                    object
SCP                                                                     object
STATION                                                                 object
LINENAME                                                                object
DIVISION                                                                object
DATE       

In [4]:
dfs.head()

,Unnamed: 0,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,0,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/01/2017,00:00:00,REGULAR,6117586,2072818
1,1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/01/2017,04:00:00,REGULAR,6117617,2072823
2,2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/01/2017,08:00:00,REGULAR,6117635,2072856
3,3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/01/2017,12:00:00,REGULAR,6117746,2072972
4,4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/01/2017,16:00:00,REGULAR,6118002,2073040


In [5]:
def preprocess(df):
    
    # STANDARDIZE COL NAMES
    df.columns = df.columns.str.strip()
        
    # Create DATETIME column from 'DATE' and 'TIME' cols
    df['DATETIME'] = pd.to_datetime(df.DATE + " " + df.TIME, 
                                    format="%m/%d/%Y %H:%M:%S")
    
    # Create 'DOW' (Day of Week) col from 'DATETIME'
    df['DOW'] = df[['DATETIME']]\
    .apply(lambda x: datetime.datetime.strftime(x['DATETIME'], '%A'), axis=1)
       
    # DROP UNNECESSARY COLUMNS
    df = df.drop(['LINENAME', 'DIVISION', 'DATE','Unnamed: 0'], axis=1) 

    # Remove non-REGULAR values from 'DESC'
    df = df.drop(df.loc[df.DESC != 'REGULAR'].index)
    df = df.drop(['DESC', 'TIME'], axis=1) 
    
    return df


In [6]:
#BREAKPOINT

df_2 = preprocess(dfs)
df_2.head()

,C/A,UNIT,SCP,STATION,ENTRIES,EXITS,DATETIME,DOW
0,A002,R051,02-00-00,59 ST,6117586,2072818,2017-04-01 00:00:00,Saturday
1,A002,R051,02-00-00,59 ST,6117617,2072823,2017-04-01 04:00:00,Saturday
2,A002,R051,02-00-00,59 ST,6117635,2072856,2017-04-01 08:00:00,Saturday
3,A002,R051,02-00-00,59 ST,6117746,2072972,2017-04-01 12:00:00,Saturday
4,A002,R051,02-00-00,59 ST,6118002,2073040,2017-04-01 16:00:00,Saturday


In [7]:
# Drop Nulls created by previous cell
print("prior shape: ", df_2.shape)
df_2.dropna(how='any', inplace=True)
print("post shape: ", df_2.shape)

prior shape:  (2448638, 8)
post shape:  (2448638, 8)


In [8]:
# Check for duplicates for a given turnstile for a given datetime
(df_2
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATETIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(5)

# DROP DUPLICATES
print("prior shape: ", df_2.shape)
df_2.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATETIME"], inplace=True)
print("post shape: ", df_2.shape)

prior shape:  (2448638, 8)
post shape:  (2448638, 8)


In [9]:
df_2 = df_2.sort_values(['STATION', 'C/A', 'UNIT', 'SCP','DATETIME'])

In [10]:
df_2.head()

,C/A,UNIT,SCP,STATION,ENTRIES,EXITS,DATETIME,DOW
29335,H007,R248,00-00-00,1 AV,12184694,13417252,2017-04-01 00:00:00,Saturday
29339,H007,R248,00-00-00,1 AV,12186126,13419539,2017-04-01 16:00:00,Saturday
29340,H007,R248,00-00-00,1 AV,12186780,13420833,2017-04-01 20:00:00,Saturday
29341,H007,R248,00-00-00,1 AV,12187300,13421711,2017-04-02 00:00:00,Sunday
29342,H007,R248,00-00-00,1 AV,12187444,13422048,2017-04-02 04:00:00,Sunday


In [11]:
# Create diff columns to take entries/exits for given time interval

df_2['entry_diff'] = df_2.groupby(['STATION', 'C/A', 'UNIT', 'SCP'], as_index=False)['ENTRIES'].diff().tolist()
df_2['exit_diff'] = df_2.groupby(['STATION', 'C/A', 'UNIT', 'SCP'], as_index=False)['EXITS'].diff().tolist()

In [12]:
df_2.head()

,C/A,UNIT,SCP,STATION,ENTRIES,EXITS,DATETIME,DOW,entry_diff,exit_diff
29335,H007,R248,00-00-00,1 AV,12184694,13417252,2017-04-01 00:00:00,Saturday,NaN,NaN
29339,H007,R248,00-00-00,1 AV,12186126,13419539,2017-04-01 16:00:00,Saturday,1432.0,2287.0
29340,H007,R248,00-00-00,1 AV,12186780,13420833,2017-04-01 20:00:00,Saturday,654.0,1294.0
29341,H007,R248,00-00-00,1 AV,12187300,13421711,2017-04-02 00:00:00,Sunday,520.0,878.0
29342,H007,R248,00-00-00,1 AV,12187444,13422048,2017-04-02 04:00:00,Sunday,144.0,337.0


In [13]:
# Only used for multiple years of data
#df_2['YEAR'] = df_2['DATETIME'].transform(lambda x: x.year)
#df_2['MONTH'] = df_2['DATETIME'].transform(lambda x: x.month)
#df_2.groupby(['YEAR', 'MONTH']).count()

In [14]:
# Drop Nulls created by previous groupby().diff()
print("prior shape: ", df_2.shape)
df_2.dropna(how='any', inplace=True)
print("post shape: ", df_2.shape)

prior shape:  (2448638, 10)
post shape:  (2443903, 10)


In [15]:
# ASSUMPTION: All negative actuals for entries and exits will be dropped from dataset

len_entry_diffs = df_2['entry_diff'].shape[0]
len_exit_diffs = df_2['exit_diff'].shape[0]

negative_entries = df_2[df_2['entry_diff'] < 0].shape[0] / len_entry_diffs
negative_exits = df_2[df_2['exit_diff'] < 0].shape[0] / len_exit_diffs

print('negative_entries: ', negative_entries)
print('negative_exits: ', negative_exits)

## DROP NEGATIVES
print("prior shape: ", df_2.shape)
df_2 = df_2.loc[ (df_2['entry_diff'] > 0) & (df_2['exit_diff'] > 0) ]
print("post shape: ", df_2.shape)

negative_entries:  0.007851784624839856
negative_exits:  0.006221196176771336
prior shape:  (2443903, 10)
post shape:  (1996409, 10)


In [16]:
# Create Outlier Threshold based on total seconds in a day
threshold_entry = 86400
threshold_exit = 86400

## DROP OUTLIERS
print("prior shape: ", df_2.shape)
df_2 = df_2[(df_2['entry_diff'] < threshold_entry) & (df_2['exit_diff'] < threshold_exit)]
print("post shape: ", df_2.shape)

prior shape:  (1996409, 10)
post shape:  (1996368, 10)


In [17]:
# Create 'TOTAL_TRAFFIC" = exits + entries. This will be our ranking metric

df_2['TOTAL_TRAFFIC'] = df_2['entry_diff'] + df_2['exit_diff']
print("post shape: ", df_2.shape)
df_2.sort_values('TOTAL_TRAFFIC', ascending=False).head()

post shape:  (1996368, 11)


,C/A,UNIT,SCP,STATION,ENTRIES,EXITS,DATETIME,DOW,entry_diff,exit_diff,TOTAL_TRAFFIC
110690,PTH02,R544,00-00-05,HARRISON,92324,7364,2017-06-06 08:27:27,Tuesday,65925.0,5782.0,71707.0
114536,PTH07,R550,00-02-00,CITY / BUS,4182,27701,2017-06-16 15:09:35,Friday,4180.0,27697.0,31877.0
114450,PTH07,R550,00-02-00,CITY / BUS,4182,27701,2017-06-21 08:47:35,Wednesday,4178.0,27477.0,31655.0
114458,PTH07,R550,00-02-00,CITY / BUS,4182,27701,2017-06-22 14:11:35,Thursday,4176.0,27399.0,31575.0
114465,PTH07,R550,00-02-00,CITY / BUS,4182,27701,2017-06-23 15:23:35,Friday,4174.0,27341.0,31515.0


In [22]:
# Force time intervals to 4-hour frequency
df_2['TIME'] = df_2['DATETIME'].apply(lambda x: x.time())
df_2 = df_2.groupby(['C/A',
                     'UNIT', 
                     'SCP', 
                     'STATION', 
                     pd.Grouper(key='DATETIME', freq='4H'), 
                     'DOW', 
                     'TIME'])\
            .sum()\
            .reset_index()

# Shift time intervals back 4 hours to make DOW accurate reading of day
df_2['DATETIME'] = df_2['DATETIME'].apply(lambda x: x - datetime.timedelta(hours=4))

print("UNIQUE TIMES: ", df_2['TIME'].unique())

UNIQUE TIMES:  [datetime.time(4, 0) datetime.time(8, 0) datetime.time(12, 0)
 datetime.time(16, 0) datetime.time(20, 0) datetime.time(0, 0)]


In [29]:
# taken from preprocess()
df_2['DOW'] = df_2['DATETIME'].apply(lambda x: datetime.datetime.strftime(x, '%A'))

In [39]:
#sanity check
#df_2.groupby([pd.Grouper(key='DATETIME', freq='1D'), 'DOW', 'STATION'])\
#    .sum()\
#    .reset_index()\
#    .groupby('DOW')\
#    .mean()

In [40]:
# We only care about TOTAL_TRAFFIC
df_2.drop(['ENTRIES', 'EXITS', 'entry_diff', 'exit_diff'], axis=1, inplace=True)
df_2.head()

,C/A,UNIT,SCP,STATION,DATETIME,DOW,TIME,TOTAL_TRAFFIC
0,A002,R051,02-00-00,59 ST,2017-04-01 00:00:00,Saturday,04:00:00,36.0
1,A002,R051,02-00-00,59 ST,2017-04-01 04:00:00,Saturday,08:00:00,51.0
2,A002,R051,02-00-00,59 ST,2017-04-01 08:00:00,Saturday,12:00:00,227.0
3,A002,R051,02-00-00,59 ST,2017-04-01 12:00:00,Saturday,16:00:00,324.0
4,A002,R051,02-00-00,59 ST,2017-04-01 16:00:00,Saturday,20:00:00,408.0


In [41]:
#sanity check

#from copy import deepcopy
#traffic_by_station = deepcopy(df_2)
#traffic_by_station['DATE'] = traffic_by_station['DATETIME'].apply(lambda x: x.date())

In [42]:
#sanity check

#tmp = traffic_by_station.groupby(['STATION','DATE'],as_index=False).sum()
#tmp.groupby('STATION', as_index=False).mean().sort_values('TOTAL_TRAFFIC',ascending=False).head(10)

In [43]:
#sanity check
#times_by_traffic = df_2.groupby('TIME').sum()

In [44]:
#sanity check 
#times_by_traffic.sort_values('TOTAL_TRAFFIC',ascending=False)

In [47]:
from difflib import SequenceMatcher
from copy import deepcopy
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [52]:
# Look for stations that are the same but named slightly differently
cleaned_df = deepcopy(df_2)
df_work = cleaned_df[cleaned_df['STATION'].str.contains('-') & cleaned_df['STATION'].str.contains('ST')]
stations = df_work.STATION.unique()
similar_pairs = []
for i,s in enumerate(stations):
    for j in range(i,len(stations)):
        if not i == j:
            sim = similar(sorted(s), sorted(stations[j]))
            if sim > 0.9:
                similar_pairs.append((stations[i],stations[j]))
similar_pairs

[('86 ST-2 AVE', '96 ST-2 AVE')]

In [49]:
#NOTE: 'replaces' is based on output from similar_pairs
#more years == more naming conflicts

def norm_names(x):
    replaces = {'TIMES SQ-42 ST':'42 ST-TIMES SQ',
                'GRD CNTRL-42 ST':'42 ST-GRD CNTRL',
                '4AV-9 ST':'4 AV-9 ST',
                '110 ST-CATHEDRL':'CATHEDRL-110 ST',
                '68ST-HUNTER CO':'68ST-HUNTER COL',
                '149 ST-3 AVE':'3 AV-149 ST'
               }
    if x in replaces:
        return replaces[x]
    else:
        return x

In [50]:
# Normalize station names which are different but referring to the same place

df_2['STATION'] = df_2['STATION'].apply(lambda x: norm_names(x))

In [51]:
# Load into "merge_weather.ipynb"
# Load into "benson_viz.ipynb"
df_2.to_csv('./prepped_data_2015-2017_presummer.csv')